Start



In [ ]:

%load_ext autoreload
%autoreload 2

# Import some libraries

In [ ]:

import os
import sys
import lmdb
import pickle
import numpy as np

# Cloning from github (Generative Models)



In [ ]:
!git clone https://github.com/kkahloots/Generative_Models.git # this is for loading git with correct brach



Cloning into 'Generative_Models'...
remote: Enumerating objects: 15891, done.
remote: Total 15891 (delta 0), reused 0 (delta 0), pack-reused 15891
Receiving objects: 100% (15891/15891), 733.46 MiB | 41.90 MiB/s, done.
Resolving deltas: 100% (10845/10845), done.
Checking out files: 100% (1697/1697), done.


# Cloning from github (LMDB_Datasets)

In [ ]:
!cd Generative_Models
!git clone https://github.com/azeghost/LMDB_Datasets # this is for loading git with correct brach

Cloning into 'LMDB_Datasets'...
remote: Enumerating objects: 52, done.
remote: Counting objects: 100% (52/52), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 266 (delta 26), reused 33 (delta 14), pack-reused 214
Receiving objects: 100% (266/266), 407.93 MiB | 46.49 MiB/s, done.
Resolving deltas: 100% (123/123), done.


In [ ]:
!ls data/.CBSD68/01

ls: cannot access 'data/.CBSD68/01': No such file or directory


In [ ]:
!ls Generative_Models/data/.CBSD68/

01  CBSD68.tar.bz


# LMDB transformer

In [ ]:
!pip install colorlog

In [ ]:
from colorlog import ColoredFormatter




**LMDB Transformer **

In [ ]:
!cp -r LMDB_Datasets/transformation/ Generative_Models/

In [ ]:
import os
os.chdir('Generative_Models')
os.getcwd()

'/content/Generative_Models'

In [ ]:
import tensorflow as tf

In [ ]:
image_size_original=(481, 321, 3)
scale = 3
image_size = list(map(lambda x: x//scale , image_size_original[:-1])) + [image_size_original[-1]]
image_size = (*image_size,)

In [ ]:
inputs_shape = list(map(lambda x: x//2 , image_size[:-1])) + [image_size[-1]]
inputs_shape = (*inputs_shape,)

In [ ]:
shrink_fn = lambda image: tf.image.resize(image, inputs_shape[:-1]).numpy()

In [ ]:
from LMDB_Datasets.transformation.lmdb_transformer import SRLmdbTransformer

In [ ]:
!ls 

data	    LICENSE	   README.md	     training
evaluation  LMDB_Datasets  requirements.txt  transformation
graphs	    notebooks	   statistical	     utils


In [ ]:
images_dir = 'data/.CBSD68' #Folder to images ( without name of the folder which we moved images before)
validation_percentage = 30
valid_format = 'png'

In [ ]:
transformer = SRLmdbTransformer(image_dir = images_dir, trans_func=shrink_fn,
                              validation_pct = validation_percentage, valid_image_formats = valid_format)

# Evaluating numbers of training and validation sets

In [ ]:

val_num_images = 0
tra_num_images = 0
for key in transformer.image_lists.keys():
    tra_num_images += len(transformer.image_lists[key]['training'])
    val_num_images += len(transformer.image_lists[key]['validation'])
    
print(tra_num_images, val_num_images)

47 21


# Getting labels by filename

In [ ]:
from LMDB_Datasets.datasets_customed.label_by_filename import get_label_by_filename


In [ ]:
def get_label_by_filename(img_path):
    name, _ = os.path.splitext(img_path)
    vid_img_arr = name.split(sep=os.sep)[-2:]
    return {'dir': (vid_img_arr)[0], 'filename': (vid_img_arr)[1]}

# Transforming images to LMDB

In [ ]:
lmdb_dir = 'data/.CBSD68_low2high_LMDB'
lmdb_dir

'data/.CBSD68_low2high_LMDB'

In [ ]:
#!mv data/.CBSD68/02 data/.CBSD68/a

In [ ]:
!ls data/.CBSD68/a

ls: cannot access 'data/.CBSD68/a': No such file or directory


In [ ]:
transformer.transform_store(labels_fn=get_label_by_filename,image_dir=images_dir, lmdb_dir = lmdb_dir
           ,category='training',target_size=(481, 321),color_mode='rgb')


Total number of imgs for catagory 47
Storing 47data/.CBSD68_low2high_LMDB/into _training from folder 01


In [ ]:
transformer.transform_store(labels_fn=get_label_by_filename,image_dir=images_dir, lmdb_dir = lmdb_dir
           ,category='validation',target_size=(481, 321),color_mode='rgb')

Total number of imgs for catagory 21
Storing 21data/.CBSD68_low2high_LMDB/into _validation from folder 01


# Saving meta data of dataset

In [ ]:
!touch ../Generative_Models/data/.CBSD68_low2high_LMDB/meta_info.json #creating a json file

In [ ]:
!ls ../Generative_Models/data/.CBSD68_low2high_LMDB

meta_info.json	_training  _validation


In [ ]:

info = {
    'Dataset_name': "CBSD68",
    'Dataset_size': val_num_images + tra_num_images,
    'val_num_images':val_num_images,
    'tra_num_images' : tra_num_images
}

transformer.save_metadata('../Generative_Models/data/.CBSD68_low2high_LMDB',info)

In [ ]:
transformer.get_metadata('../Generative_Models/data/.CBSD68_low2high_LMDB') #checking whether it is working or not

{'Dataset_name': 'CBSD68',
 'Dataset_size': 68,
 'tra_num_images': 47,
 'val_num_images': 21}

In [ ]:
!ls -l /content/Generative_Models/data/.CBSD68_low2high_LMDB

total 12
-rw-r--r-- 1 root root  102 Nov  4 21:21 meta_info.json
drwxr-xr-x 2 root root 4096 Nov  4 21:21 _training
drwxr-xr-x 2 root root 4096 Nov  4 21:21 _validation


# I tried to read LMDB, but..

In [ ]:
import lmdb
import pickle

def read_many_lmdb(lmdb_dir, num_images):

    images, labels = [], {}
    env = lmdb.open(lmdb_dir, readonly=True)

    # Start a new read transaction
    with env.begin() as txn:
        # Read all images in one single transaction, with one lock
        # We could split this up into multiple transactions if needed
        for image_id in range(num_images):
            data = txn.get(f"{image_id:08}".encode("ascii"))
            # Remember that it's a CIFAR_Image object 
            # that is stored as the value
            dataset = pickle.loads(data)
            images.append(dataset.get_image())
            
            labels_list = [attr for attr in dir(dataset) if not callable(getattr(dataset, attr)) and (not attr.startswith("__")) and 
                           (not attr in ['image','channels',  'size'] )]

            for label in labels_list:
                # _lab = {label: eval(f'dataset.{label}')}
#                 print({label: eval(f'dataset.{label}')})
                # labels = {**labels, **_lab}
                if label in labels:
                    labels[label].append(eval(f'dataset.{label}'))
                else:
                    labels = {label: [eval(f'dataset.{label}')] }
                
    env.close()
    return {'images': images, **labels}

I don't know why it gives the following error.

In [ ]:
ds = read_many_lmdb('/content/Generative_Models/CBSD68_LMDB/_training', 5)

Error: ignored

# Zipping

In [ ]:
!ls data/.CBSD68_low2high_LMDB/_training/

data.mdb  lock.mdb


In [ ]:
!tar -cvjf CBSD68_low2high_LMDB.tar.bz data/.CBSD68_low2high_LMDB

data/.CBSD68_low2high_LMDB/
data/.CBSD68_low2high_LMDB/_training/
data/.CBSD68_low2high_LMDB/_training/lock.mdb
data/.CBSD68_low2high_LMDB/_training/data.mdb
data/.CBSD68_low2high_LMDB/meta_info.json
data/.CBSD68_low2high_LMDB/_validation/
data/.CBSD68_low2high_LMDB/_validation/lock.mdb
data/.CBSD68_low2high_LMDB/_validation/data.mdb


In [ ]:
!ls CBSD68_low2high_LMDB.tar.bz -l

-rw-r--r-- 1 root root 21074574 Nov  4 21:22 CBSD68_low2high_LMDB.tar.bz


In [ ]:
!zip -r LMDB_Datasets.zip  LMDB_Datasets/

  adding: LMDB_Datasets/ (stored 0%)
  adding: LMDB_Datasets/requirements.txt (deflated 47%)
  adding: LMDB_Datasets/datasets_customed/ (stored 0%)
  adding: LMDB_Datasets/datasets_customed/label_by_filename.py (deflated 58%)
  adding: LMDB_Datasets/datasets_customed/__pycache__/ (stored 0%)
  adding: LMDB_Datasets/datasets_customed/__pycache__/label_by_filename.cpython-36.pyc (deflated 33%)
  adding: LMDB_Datasets/.git/ (stored 0%)
  adding: LMDB_Datasets/.git/info/ (stored 0%)
  adding: LMDB_Datasets/.git/info/exclude (deflated 28%)
  adding: LMDB_Datasets/.git/branches/ (stored 0%)
  adding: LMDB_Datasets/.git/packed-refs (deflated 11%)
  adding: LMDB_Datasets/.git/config (deflated 31%)
  adding: LMDB_Datasets/.git/logs/ (stored 0%)
  adding: LMDB_Datasets/.git/logs/HEAD (deflated 27%)
  adding: LMDB_Datasets/.git/logs/refs/ (stored 0%)
  adding: LMDB_Datasets/.git/logs/refs/heads/ (stored 0%)
  adding: LMDB_Datasets/.git/logs/refs/heads/master (deflated 27%)
  adding: LMDB_Datasets

In [ ]:
!ls

CBSD68_low2high_LMDB.tar.bz  LICENSE	    requirements.txt  utils
data			     LMDB_Datasets  statistical
evaluation		     notebooks	    training
graphs			     README.md	    transformation


In [ ]:
!tar -cvjf LMDB_Datasets.tar.bz LMDB_Datasets

LMDB_Datasets/
LMDB_Datasets/requirements.txt
LMDB_Datasets/datasets_customed/
LMDB_Datasets/datasets_customed/label_by_filename.py
LMDB_Datasets/datasets_customed/__pycache__/
LMDB_Datasets/datasets_customed/__pycache__/label_by_filename.cpython-36.pyc
LMDB_Datasets/.git/
LMDB_Datasets/.git/info/
LMDB_Datasets/.git/info/exclude
LMDB_Datasets/.git/branches/
LMDB_Datasets/.git/packed-refs
LMDB_Datasets/.git/config
LMDB_Datasets/.git/logs/
LMDB_Datasets/.git/logs/HEAD
LMDB_Datasets/.git/logs/refs/
LMDB_Datasets/.git/logs/refs/heads/
LMDB_Datasets/.git/logs/refs/heads/master
LMDB_Datasets/.git/logs/refs/remotes/
LMDB_Datasets/.git/logs/refs/remotes/origin/
LMDB_Datasets/.git/logs/refs/remotes/origin/HEAD
LMDB_Datasets/.git/objects/
LMDB_Datasets/.git/objects/info/
LMDB_Datasets/.git/objects/pack/
LMDB_Datasets/.git/objects/pack/pack-5c407f99e1c4b4d8f8c931715eab88776f4e9c67.idx
LMDB_Datasets/.git/objects/pack/pack-5c407f99e1c4b4d8f8c931715eab88776f4e9c67.pack
LMDB_Datasets/.git/description